# Quantitative Momementum Strategy

##  Library Import

In [27]:
import numpy as np
import pandas as pd
import requests
import math
from scipy.stats import percentileofscore as score
import xlsxwriter

## Import list of stocks

In [28]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making the first API call

In [29]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data =  requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2881472976031,
 'week52high': 183.43,
 'week52low': 123.37,
 'week52highSplitAdjustOnly': 191.11,
 'week52lowSplitAdjustOnly': 126,
 'week52change': 0.35523560211847105,
 'sharesOutstanding': 16423759818,
 'float': 0,
 'avg10Volume': 90671739,
 'avg30Volume': 96788187,
 'day200MovingAvg': 165.5,
 'day50MovingAvg': 173.58,
 'employees': 153503,
 'ttmEPS': 6.23,
 'ttmDividendRate': 0.9008024261162979,
 'dividendYield': 0.005211002500056689,
 'nextDividendDate': '',
 'exDividendDate': '2022-02-02',
 'nextEarningsDate': '2022-04-17',
 'peRatio': 28.022480561592776,
 'beta': 1.293712438010208,
 'maxChangePercent': 65.23533189528102,
 'year5ChangePercent': 4.160711128334318,
 'year2ChangePercent': 1.709696918028221,
 'year1ChangePercent': 0.3738952192162013,
 'ytdChangePercent': -0.03210330546635237,
 'month6ChangePercent': 0.2214758642756876,
 'month3ChangePercent': 0.0002927375047130062,
 'month1ChangePercent': 0.05543851194517687,
 'day30ChangePe

In [30]:
data['year1ChangePercent']

0.3738952192162013

## Executing a Bacth API call and Building the DataFrame

In [31]:
def chunks(lst, n):
    """ Yield succesive n-sized chunks from lst"""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    # print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-year Price Return', 'Number of Shares to Buy']

In [135]:
final_dataframe = pd.DataFrame(columns=my_columns)
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(pd.Series(
        [
            symbol,
            data[symbol]['price'],
            data[symbol]['stats']['year1ChangePercent'],
            'N/A'
        ], index=my_columns),
        ignore_index=True
        )
final_dataframe

,Ticker,Price,One-year Price Return,Number of Shares to Buy
0,A,139.730,0.0569202,N/A
1,AAL,17.225,-0.292663,N/A
2,AAP,235.400,0.220154,N/A
3,AAPL,178.180,0.361458,N/A
4,ABBV,180.660,0.757162,N/A
5,ABC,166.360,0.425526,N/A
6,ABMD,318.420,-0.00981162,N/A
7,ABT,124.885,0.0429602,N/A
8,ACN,355.212,0.231136,N/A
9,ADBE,471.360,-0.0837731,N/A


## Removing Low-Momentum Stocks

In [33]:
final_dataframe.sort_values('One-year Price Return', ascending=False, inplace=True)
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace=True)
final_dataframe

,index,Ticker,Price,One-year Price Return,Number of Shares to Buy
0,273,LB,83.41,2.303,N/A
1,147,DVN,62.42,1.77089,N/A
2,42,APA,43.00,1.3538,N/A
3,89,CF,108.25,1.33211,N/A
4,312,MOS,73.99,1.29664,N/A
5,315,MRO,25.75,1.27684,N/A
6,355,OXY,58.75,1.24042,N/A
7,109,COP,102.53,0.936587,N/A
8,342,NUE,151.57,0.904054,N/A
9,174,FANG,139.97,0.826731,N/A


## Calculating the Number od Shares to Buy

In [34]:
def portfolio_input():
    global portfolio_size
    while True:
        try:
            portfolio_size = float(input('Enter the value of your portfolio: '))
            break
        except ValueError:
                    print("Thats not  number! \nPlease try again")



In [35]:
portfolio_input()

Enter the value of your portfolio: 10000000


In [36]:
position_size = portfolio_size / len(final_dataframe.index)
for i in range(0, len(final_dataframe)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i, 'Price'])

final_dataframe

C:\WPy64-3720\python-3.7.2.amd64\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,index,Ticker,Price,One-year Price Return,Number of Shares to Buy
0,273,LB,83.41,2.303,2397
1,147,DVN,62.42,1.77089,3204
2,42,APA,43.00,1.3538,4651
3,89,CF,108.25,1.33211,1847
4,312,MOS,73.99,1.29664,2703
5,315,MRO,25.75,1.27684,7766
6,355,OXY,58.75,1.24042,3404
7,109,COP,102.53,0.936587,1950
8,342,NUE,151.57,0.904054,1319
9,174,FANG,139.97,0.826731,1428


## Building a Better and more Realistic momentum Strategy

In [121]:
hqm_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
    'HQM Score'
]

hqm_dataframe = pd.DataFrame(columns=hqm_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(pd.Series(
        [
            symbol,
            data[symbol]['price'],
            'N/A',
            data[symbol]['stats']['year1ChangePercent'],
            'N/A',
            data[symbol]['stats']['month6ChangePercent'],
            'N/A',
            data[symbol]['stats']['month3ChangePercent'],
            'N/A',
            data[symbol]['stats']['month1ChangePercent'],
            'N/A',
            'N/A'
        ],
        index=hqm_columns),
        ignore_index=True
        )
        hqm_dataframe.replace(to_replace=[None], value=0, inplace=True)

hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,138.93,N/A,0.057945,N/A,-0.125265,N/A,-0.064185,N/A,0.045198,N/A,N/A
1,AAL,17.30,N/A,-0.294414,N/A,-0.160475,N/A,-0.122160,N/A,0.326819,N/A,N/A
2,AAP,222.52,N/A,0.228054,N/A,0.026057,N/A,-0.083062,N/A,0.090088,N/A,N/A
3,AAPL,177.78,N/A,0.362601,N/A,0.206733,N/A,0.001140,N/A,0.084077,N/A,N/A
4,ABBV,173.53,N/A,0.742726,N/A,0.610993,N/A,0.298054,N/A,0.164029,N/A,N/A
5,ABC,170.14,N/A,0.429560,N/A,0.396249,N/A,0.233005,N/A,0.126717,N/A,N/A
6,ABMD,321.66,N/A,-0.009849,N/A,-0.069081,N/A,-0.005547,N/A,0.063523,N/A,N/A
7,ABT,129.41,N/A,0.042753,N/A,0.055206,N/A,-0.090935,N/A,0.039239,N/A,N/A
8,ACN,350.41,N/A,0.225307,N/A,0.054825,N/A,-0.077962,N/A,0.122000,N/A,N/A
9,ADBE,464.91,N/A,-0.084429,N/A,-0.220161,N/A,-0.115637,N/A,0.034263,N/A,N/A


## Calculating Momentum Percentiles

In [137]:
time_periods =[
    'One-Year',
    'Six-Month',
    'Three-Month',
    'One-Month'
    ]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        hqm_dataframe.loc[row, percentile_col] = score(hqm_dataframe[change_col], hqm_dataframe.loc[row, change_col])/100
        
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,MOS,73.08,273,1.358044,0.96,0.850765,1.00,0.798348,1.00,0.173933,0.84,0.984032
1,ABBV,173.53,115,0.742726,0.76,0.610993,0.80,0.298054,0.76,0.164029,0.82,0.961078
2,NUE,151.30,132,0.932820,0.88,0.548149,0.76,0.328255,0.78,0.150989,0.62,0.955589
3,MCK,322.48,62,0.680733,0.70,0.616321,0.82,0.285906,0.72,0.155849,0.70,0.952096
4,VLO,106.12,188,0.481597,0.40,0.413616,0.58,0.290985,0.74,0.237619,0.96,0.949601
5,LLY,309.39,64,0.750135,0.82,0.346401,0.42,0.195484,0.36,0.180750,0.92,0.939621
6,MPC,86.69,230,0.677637,0.68,0.379931,0.50,0.220139,0.48,0.153172,0.66,0.928144
7,CF,106.48,187,1.331155,0.92,0.777314,0.92,0.532524,0.96,0.103033,0.24,0.926647
8,EXC,51.03,391,0.624027,0.60,0.481835,0.64,0.250773,0.62,0.141385,0.50,0.924651
9,ADM,95.46,209,0.674639,0.66,0.530357,0.70,0.357397,0.84,0.120757,0.38,0.923653


## Calculating the HQM Score

In [123]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,138.93,N/A,0.057945,0.45509,-0.125265,0.191617,-0.064185,0.417166,0.045198,0.459082,0.380739
1,AAL,17.30,N/A,-0.294414,0.0538922,-0.160475,0.133733,-0.122160,0.271457,0.326819,0.992016,0.362774
2,AAP,222.52,N/A,0.228054,0.712575,0.026057,0.512974,-0.083062,0.371257,0.090088,0.670659,0.566866
3,AAPL,177.78,N/A,0.362601,0.852295,0.206733,0.824351,0.001140,0.592814,0.084077,0.644711,0.728543
4,ABBV,173.53,N/A,0.742726,0.968064,0.610993,0.978044,0.298054,0.972056,0.164029,0.926148,0.961078
5,ABC,170.14,N/A,0.429560,0.878244,0.396249,0.944112,0.233005,0.934132,0.126717,0.812375,0.892216
6,ABMD,321.66,N/A,-0.009849,0.325349,-0.069081,0.323353,-0.005547,0.552894,0.063523,0.546906,0.437126
7,ABT,129.41,N/A,0.042753,0.423154,0.055206,0.57485,-0.090935,0.353293,0.039239,0.429142,0.44511
8,ACN,350.41,N/A,0.225307,0.702595,0.054825,0.570858,-0.077962,0.383234,0.122000,0.796407,0.613273
9,ADBE,464.91,N/A,-0.084429,0.233533,-0.220161,0.0798403,-0.115637,0.283433,0.034263,0.401198,0.249501


## Selecting the 50 Best Momentum Stocks

In [124]:
hqm_dataframe.sort_values('HQM Score', ascending = False, inplace=True)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(inplace=True, drop=True)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,MOS,73.08,N/A,1.358044,0.994012,0.850765,1,0.798348,1,0.173933,0.942116,0.984032
1,ABBV,173.53,N/A,0.742726,0.968064,0.610993,0.978044,0.298054,0.972056,0.164029,0.926148,0.961078
2,NUE,151.30,N/A,0.932820,0.984032,0.548149,0.974052,0.328255,0.976048,0.150989,0.888224,0.955589
3,MCK,322.48,N/A,0.680733,0.956088,0.616321,0.98004,0.285906,0.966068,0.155849,0.906188,0.952096
4,VLO,106.12,N/A,0.481597,0.902196,0.413616,0.946108,0.290985,0.97006,0.237619,0.98004,0.949601
5,LLY,309.39,N/A,0.750135,0.974052,0.346401,0.922156,0.195484,0.906188,0.180750,0.956088,0.939621
6,MPC,86.69,N/A,0.677637,0.954092,0.379931,0.934132,0.220139,0.926148,0.153172,0.898204,0.928144
7,CF,106.48,N/A,1.331155,0.99002,0.777314,0.992016,0.532524,0.996008,0.103033,0.728543,0.926647
8,EXC,51.03,N/A,0.624027,0.946108,0.481835,0.96008,0.250773,0.942116,0.141385,0.850299,0.924651
9,ADM,95.46,N/A,0.674639,0.952096,0.530357,0.968064,0.357397,0.982036,0.120757,0.792415,0.923653


## Calculating the Number of Shares to Buy

In [125]:
portfolio_input()

Enter the value of your portfolio: 1000000


In [126]:
position_size = float(portfolio_size) / len(hqm_dataframe.index)
for i in hqm_dataframe.index:
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/hqm_dataframe.loc[i, 'Price']) 
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,MOS,73.08,273,1.358044,0.994012,0.850765,1,0.798348,1,0.173933,0.942116,0.984032
1,ABBV,173.53,115,0.742726,0.968064,0.610993,0.978044,0.298054,0.972056,0.164029,0.926148,0.961078
2,NUE,151.30,132,0.932820,0.984032,0.548149,0.974052,0.328255,0.976048,0.150989,0.888224,0.955589
3,MCK,322.48,62,0.680733,0.956088,0.616321,0.98004,0.285906,0.966068,0.155849,0.906188,0.952096
4,VLO,106.12,188,0.481597,0.902196,0.413616,0.946108,0.290985,0.97006,0.237619,0.98004,0.949601
5,LLY,309.39,64,0.750135,0.974052,0.346401,0.922156,0.195484,0.906188,0.180750,0.956088,0.939621
6,MPC,86.69,230,0.677637,0.954092,0.379931,0.934132,0.220139,0.926148,0.153172,0.898204,0.928144
7,CF,106.48,187,1.331155,0.99002,0.777314,0.992016,0.532524,0.996008,0.103033,0.728543,0.926647
8,EXC,51.03,391,0.624027,0.946108,0.481835,0.96008,0.250773,0.942116,0.141385,0.850299,0.924651
9,ADM,95.46,209,0.674639,0.952096,0.530357,0.968064,0.357397,0.982036,0.120757,0.792415,0.923653


## Formatting Excel Output

In [131]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index=False)

## Creating the Formats for the .xlsx File

In [132]:
background_color = '#0a023'
font_color = '#ffffff'

string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

dollar_format = writer.book.add_format(
    {
        'num_format': '$0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

integer_format = writer.book.add_format(
    {
        'num_format': '0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)
percent_format = writer.book.add_format(
    {
        'num_format': '0.0%',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

In [133]:
column_formats = {
    'A': ['Ticker', string_format],
    'B': ['Price', dollar_format],
    'C': ['Num of Shares to Buy', integer_format],
    'D': ['One-Year Price Return', percent_format],
    'E': ['One-Year Return Percentile', percent_format],
    'F': ['Six-Month Price Return', percent_format],
    'G': ['Six-Month Return Percentile', percent_format],
    'H': ['Three-Month Price Return', percent_format],
    'I': ['Three-Month Return Percentile', percent_format],
    'J': ['One-Month Price Return', percent_format],
    'K': ['One-Month Return Percentile', percent_format],
    'L': ['HQM Score', percent_format]
}

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 22, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

## Save the Excel Output

In [134]:
writer.save()